# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [88]:
#!pip install python-dotenv
#!pip install openai

In [89]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['GROQ_API_KEY']



Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [ ]:
# 1. Definimos el modelo actualizado
# 'llama-3.1-8b-instant' es el reemplazo directo y actual.
llm_model = "llama-3.1-8b-instant"

# 2. Re-configuramos el cliente de Groq (para get_completion)
from openai import OpenAI
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

# 3. Re-configuramos el objeto de LangChain (para chat.invoke)
from langchain_groq import ChatGroq
chat = ChatGroq(model_name="llama-3.1-8b-instant")


## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [91]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content


In [92]:
print(get_completion("What is 1+1?"))

1 + 1 = 2.


In [93]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [94]:
style = """American English \
in a calm and respectful tone
"""

In [95]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [96]:
response = get_completion(prompt)
print(response)

Here's the translation:

"I'm extremely frustrated that my blender lid flew off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now."


In [97]:
response

'Here\'s the translation:\n\n"I\'m extremely frustrated that my blender lid flew off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn\'t cover the cost of cleaning up the mess. I could really use your help right now."'

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [98]:
#!pip install --upgrade langchain

### Model

In [99]:
from langchain_openai import ChatOpenAI

In [100]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(
    temperature=0.0, 
    model="llama-3.1-8b-instant",
    openai_api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)
chat

ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x00000232D7E4E350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000232D7E4C650>, root_client=<openai.OpenAI object at 0x00000232D7D5B7D0>, root_async_client=<openai.AsyncOpenAI object at 0x00000232D7E285D0>, model_name='llama-3.1-8b-instant', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')

### Prompt template

In [101]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [102]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)


In [103]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [104]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [105]:
customer_style = """American English \
in a calm and respectful tone
"""

In [106]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [107]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [108]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [109]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [110]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)
print(customer_response)

content='Here\'s the translation of the text into American English in a calm and respectful tone:\n\n"I\'m really frustrated that my blender lid flew off and spilled my smoothie all over my kitchen walls. To make matters worse, the warranty doesn\'t cover the cost of cleaning up the mess. I could really use your help right now."' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 117, 'total_tokens': 183, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.150248832, 'prompt_time': 0.007120915, 'completion_time': 0.101076542, 'total_time': 0.108197457}, 'model_provider': 'openai', 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'id': 'chatcmpl-3284cee2-9970-4633-b351-fd9884e92aa3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b9484-720c-7e60-bc2a-a9f20ee70af9-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_token

In [111]:
print(customer_response.content)

Here's the translation of the text into American English in a calm and respectful tone:

"I'm really frustrated that my blender lid flew off and spilled my smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now."


In [112]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [113]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [114]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [115]:
service_response = chat.invoke(service_messages)
print(service_response.content)

```Hey there customer, I be afraid the warranty don't cover the cost o' cleanin' yer kitchen, matey. It seems ye had a wee mishap with yer blender, forgettin' to put the lid on before set-tin' it sail. A bit o' carelessness, if ye don't mind me sayin' so. I be hopin' ye'll take heed o' this lesson and be more mindful o' yer kitchen contraptions in the future. Fair winds and following seas to ye, but I be afraid ye'll be footin' the bill for this one, matey!```


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [116]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [117]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [118]:
# Importaciones para LangChain v0.2 / v0.3
from langchain_classic.output_parsers import ResponseSchema, StructuredOutputParser
# Verificamos que funcione definiendo un esquema rápido
gift_schema = ResponseSchema(name="gift", description="Es un regalo?")
output_parser = StructuredOutputParser.from_response_schemas([gift_schema])

print("¡Importación exitosa!")

¡Importación exitosa!


In [119]:
from langchain_classic.output_parsers import ResponseSchema
from langchain_classic.output_parsers import StructuredOutputParser
from langchain_classic.chat_models import ChatOpenAI
from langchain_classic.prompts import ChatPromptTemplate

# 1. Definimos qué queremos extraer
monto_schema = ResponseSchema(
    name="monto",
    description="El valor monetario de la transferencia. Solo el número."
)
destinatario_schema = ResponseSchema(
    name="destinatario",
    description="El nombre de la persona o empresa que recibe el dinero."
)
banco_schema = ResponseSchema(
    name="banco",
    description="El nombre del banco destino. Si no se menciona, poner 'Desconocido'."
)

# Metemos todo en una lista
response_schemas = [monto_schema, destinatario_schema, banco_schema]

In [120]:
# 2. Creamos el parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 3. Generamos las instrucciones automáticas (el texto técnico para el prompt)
format_instructions = output_parser.get_format_instructions()

In [121]:
# 4. Definimos el mensaje del usuario
mensaje_usuario = "Le mandé 5500 pesos a Juan Perez que tiene cuenta en el Banco Galicia."

# 5. Armamos la plantilla incluyendo las instrucciones de formato
template = """
Analiza el siguiente mensaje de una transferencia bancaria y extrae la información solicitada.

Mensaje: {texto}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)
messages = prompt.format_messages(
    texto=mensaje_usuario, 
    format_instructions=format_instructions
)

# 6. Llamamos a Groq (usando el objeto 'chat' que configuramos antes)
response = chat.invoke(messages)
# 7. ¡EL PASO CLAVE! Parseamos el string a un diccionario de Python
output_dict = output_parser.parse(response.content)
# 8. Verificación de resultados
print("--- Resultado como Diccionario ---")
print(output_dict)

--- Resultado como Diccionario ---
{'monto': '5500', 'destinatario': 'Juan Perez', 'banco': 'Banco Galicia'}


#MEMORY:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
import os


# 1. Configuramos el LLM principal
llm = ChatOpenAI(
    temperature=0,
    model="llama-3.1-8b-instant",
    api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

# 2. Configuramos la Memoria de Resumen
# max_token_limit: Es el umbral. Si la charla supera esto, empieza a resumir.
# Ponemos un número bajo (ej: 100) para ver el resumen rápido.
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

# 3. Creamos la cadena
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True  # VITAL para ver el resumen en consola
)

ValueError: "ChatOpenAI" object has no field "get_num_tokens_from_messages"

In [124]:
# Paso 1: Introducción
conversation.predict(input="Hola, soy Luciano y vivo en Argentina. Trabajo como Analista Programador.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, soy Luciano y vivo en Argentina. Trabajo como Analista Programador.
AI:


NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model llama-3.1-8b-instant. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.